
# Modelado de Datos: Capa Gold en Azure Databricks

Este notebook toma los datos ya transformados de la **capa Silver** y los organiza en un modelo de datos consumible para análisis, usualmente siguiendo un enfoque de modelo estrella o constelación (star schema), y escribe los resultados en la **capa Gold** utilizando Delta Lake.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff
from delta.tables import DeltaTable

In [0]:
from typing import Optional
from pyspark.sql.functions import col, current_timestamp, expr, lit
from functools import reduce
from operator import and_

def leer_desde_silver(
    nombre_tabla: str,
    catalog_name: str = "desarrollo",
    db_silver: str = "silver_ventas",
    mode: str = "full",                 # "full" | "diff"
    ts_col: str = "fecha_carga",
    last_n_days: Optional[int] = None,  # ej. 2  -> últimos 2 días
    last_n_hours: Optional[int] = None, # ej. 6  -> últimas 6 horas
    since: Optional[str] = None,        # "YYYY-MM-DD" o "YYYY-MM-DD HH:mm:ss"
    until: Optional[str] = None,        # límite superior EXCLUSIVO
    drop_nulls: bool = True
):
    full = f"{catalog_name}.{db_silver}.{nombre_tabla}"
    if not spark.catalog.tableExists(full):
        raise ValueError(f"La tabla {full} no existe en el metastore.")

    df = spark.table(full)

    # Validar tipo timestamp
    dtype = dict(df.dtypes).get(ts_col, "").lower()
    if dtype != "timestamp":
        raise ValueError(f"La columna '{ts_col}' debe ser timestamp. Actual: {dtype}")

    if mode.lower() == "full":
        return df if not drop_nulls else df.filter(col(ts_col).isNotNull())

    # --- Diferencial por timestamp ---
    conds = []
    if drop_nulls:
        conds.append(col(ts_col).isNotNull())

    if last_n_days is not None:
        conds.append(col(ts_col) >= (current_timestamp() - expr(f"INTERVAL {int(last_n_days)} DAYS")))
    if last_n_hours is not None:
        conds.append(col(ts_col) >= (current_timestamp() - expr(f"INTERVAL {int(last_n_hours)} HOURS")))
    if since:
        conds.append(col(ts_col) >= lit(since).cast("timestamp"))
    if until:
        conds.append(col(ts_col) <  lit(until).cast("timestamp"))  # upper bound abierto

    if not conds:
        raise ValueError("En mode='diff' especifica last_n_days/last_n_hours o since/until.")

    return df.filter(reduce(and_, conds))


In [0]:
# Full
#df_full = leer_desde_silver("t_camas", db_silver="clinica_silver", mode="full")

# Diferencial: últimos 2 días por 'fecalta'
#df_diff_2d = leer_desde_silver("t_camas", "clinica_silver", mode="diff", date_col="fecalta", last_n_days=2)

# Diferencial: rango específico [2025-08-20, 2025-08-23)
#df_diff_range = leer_desde_silver("t_camas", "clinica_silver", mode="diff",
 #                                 date_col="fecalta", start="2025-08-20", end="2025-08-23")

# Si 'fecalta' es string tipo "YYYY-MM-DD HH:mm:ss", ajusta el formato si difiere:
# string_datetime_fmt="dd/MM/yyyy HH:mm:ss"


In [0]:
from pyspark.sql.functions import col, to_timestamp, when

# Leer desde silver
#df = leer_desde_silver("md_camas", "clinica_silver", mode="diff",
#                           since="2025-08-20 00:00:00", until="2025-08-23 00:00:00")
subcategoria = leer_desde_silver("md_subcategoria", db_silver="silver_ventas", mode="full")
categoria = leer_desde_silver("md_categoria", db_silver="silver_ventas", mode="full")
producto = leer_desde_silver("md_producto", db_silver="silver_ventas", mode="full")
ubigeo = leer_desde_silver("md_ubigeo", db_silver="silver_ventas", mode="full")
segmento = leer_desde_silver("md_segmento", db_silver="silver_ventas", mode="full")
mercado = leer_desde_silver("md_mercado", db_silver="silver_ventas", mode="full")
sectoreconomico = leer_desde_silver("md_sectoreconomico", db_silver="silver_ventas", mode="full")
# cliente = leer_desde_silver("md_cliente", db_silver="silver_ventas", mode="full")
vendedor = leer_desde_silver("md_vendedor", db_silver="silver_ventas", mode="full")
modalidadenvio = leer_desde_silver("md_modalidadenvio", db_silver="silver_ventas", mode="full")
modalidadventa = leer_desde_silver("md_modalidadventa", db_silver="silver_ventas", mode="full")
moneda = leer_desde_silver("md_moneda", db_silver="silver_ventas", mode="full")
mediopago = leer_desde_silver("md_mediopago", db_silver="silver_ventas", mode="full")
prioridadpedido = leer_desde_silver("md_prioridadpedido", db_silver="silver_ventas", mode="full")
pedido = leer_desde_silver("hd_pedido", db_silver="silver_ventas", mode="full")

In [0]:
from pyspark.sql.functions import col

# Realizando los JOINs de manera estructurada con ALIAS para mayor claridad
df_Dimproducto = (
    producto.alias("p")
    .join(subcategoria.alias("s"), col("p.id_subcategoria") == col("s.id_subcategoria"), "left")
    .join(categoria.alias("c"), col("s.id_categoria") == col("c.id_categoria"), "left")
    .drop(col("s.id_subcategoria"), col("s.fecha_carga"), col("s.year"), col("s.month"), col("s.day"))
    .drop(col("c.id_categoria"), col("c.fecha_carga"), col("c.year"), col("c.month"), col("c.day"))
    .drop(col("c._etl_source"), col("c._etl_batch_id"), col("c._etl_loaded_at"))
    .drop(col("s._etl_source"), col("s._etl_batch_id"), col("s._etl_loaded_at"))
)

# Mostrar el esquema del DataFrame resultante
df_Dimproducto.printSchema()

# Mostrar las primeras filas del DataFrame para ver la relación
df_Dimproducto.display(10)

In [0]:
# from pyspark.sql.functions import col

# # Realizando los JOINs de manera estructurada con ALIAS para mayor claridad
# df_DimCliente = (
#     cliente.alias("cl")
#     .join(ubigeo.alias("ub"), col("cl.id_ubigeo") == col("ub.id_ubigeo"), "left")
#     .join(sectoreconomico.alias("se"), col("cl.id_sectoreconomico") == col("se.id_sectoreconomico"), "left")
#     .join(mercado.alias("me"), col("cl.id_mercado") == col("me.id_mercado"), "left")
#     .join(segmento.alias("sg"), col("se.id_segmento") == col("sg.id_segmento"), "left")
#     .drop(col("se.fecha_carga"),col("ub.fecha_carga"),col("me.fecha_carga"),col("sg.fecha_carga"), col("ub.id_ubigeo"))
#     .drop(col("me.id_mercado"),col("sg.id_segmento"),col("me.fecha_carga"),col("se.id_sectoreconomico"))
#     .drop(col("ub.year"), col("ub.month"), col("ub.day"), col("se.year"), col("se.month"), col("se.day"))
#     .drop(col("me.year"), col("me.month"), col("me.day"), col("sg.year"), col("sg.month"), col("sg.day"))
#     .drop(col("ub._etl_source"), col("ub._etl_batch_id"), col("ub._etl_loaded_at"))
#     .drop(col("se._etl_source"), col("se._etl_batch_id"), col("se._etl_loaded_at"))
#     .drop(col("me._etl_source"), col("me._etl_batch_id"), col("me._etl_loaded_at"))
#     .drop(col("sg._etl_source"), col("sg._etl_batch_id"), col("sg._etl_loaded_at"))
# )

# ## from cliente cl
# #left join ubigeo ub on cl.CODUBIGEO = ub.CODUBIGEO
# #left join sectoreconomico se on cl.CODSECTECON = se.CODSECTECON
# #left join mercado me on cl.CODMRCADO = me.CODMRCADO
# #left join segmento sg on se.CODSGMNTO = sg.CODSGMNTO

# # Mostrar el esquema del DataFrame resultante
# df_DimCliente.printSchema()

# # Mostrar las primeras filas del DataFrame para ver la relación
# df_DimCliente.display(10)

In [0]:
pedido.printSchema()

In [0]:
from typing import Optional
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

def crear_dim_tiempo(
    df: DataFrame,
    date_col: str,
    *,
    start_date: Optional[str] = None,   # "YYYY-MM-DD" opcional
    end_date: Optional[str] = None,     # "YYYY-MM-DD" opcional
    date_fmt: Optional[str] = None,     # ej. "yyyy-MM-dd" si es string
    include_extras: bool = False        # agrega columnas adicionales
) -> DataFrame:
    """
    Crea dimensión tiempo a partir de un DataFrame y una columna de fecha.
    - date_col: columna en df (date/timestamp/string).
    - start_date/end_date: sobrescriben los valores por defecto (min y max de la columna).
    - date_fmt: formato si la columna es string (ej. "yyyy/MM/dd").
    - include_extras: agrega columnas útiles como semana ISO, nombre de mes, etc.

    Devuelve:
      DataFrame con columnas:
        id_tiempo (yyyymmdd int), fecha, anio, mes, dia, trimestre
      + extras si include_extras=True
    """

    # --- Normaliza a date
    if date_fmt:
        fecha_expr = F.to_date(F.col(date_col), date_fmt)
    else:
        fecha_expr = F.to_date(F.col(date_col))

    df_dates = df.select(fecha_expr.alias("_fecha_base")).where(F.col("_fecha_base").isNotNull())

    # --- Determina rango de fechas (min y max del DF si no se pasan argumentos)
    minmax = df_dates.agg(
        F.min("_fecha_base").alias("min_fecha"),
        F.max("_fecha_base").alias("max_fecha")
    ).first()

    if not minmax or minmax["min_fecha"] is None or minmax["max_fecha"] is None:
        raise ValueError(f"No se encontraron fechas válidas en la columna '{date_col}'.")

    start_date = start_date or minmax["min_fecha"].strftime("%Y-%m-%d")
    end_date = end_date or minmax["max_fecha"].strftime("%Y-%m-%d")

    # --- Genera secuencia de fechas
    df_rango = (
        df.sparkSession.createDataFrame([(1,)], ["dummy"])
        .select(F.sequence(
            F.to_date(F.lit(start_date)),
            F.to_date(F.lit(end_date)),
            F.expr("interval 1 day")
        ).alias("fechas"))
    )
    df_fechas = df_rango.select(F.explode(F.col("fechas")).alias("fecha"))

    # --- Atributos base
    df_dim = (
        df_fechas
        .select(
            F.col("fecha").cast("date").alias("fecha"),
            F.year("fecha").cast("int").alias("anio"),
            F.month("fecha").cast("int").alias("mes"),
            F.dayofmonth("fecha").cast("int").alias("dia"),
            F.quarter("fecha").cast("int").alias("trimestre")
        )
        .withColumn("id_tiempo",
            (F.col("anio") * F.lit(10000) + F.col("mes") * F.lit(100) + F.col("dia")).cast("int")
        )
    )

    # --- Extras opcionales
    if include_extras:
        df_dim = (
            df_dim
            .withColumn("anio_mes", (F.col("anio")*100 + F.col("mes")).cast("int"))
            .withColumn("semana_iso", F.weekofyear("fecha").cast("int"))
            .withColumn("dia_semana_iso", F.date_format("fecha", "u").cast("int"))  # 1=lunes..7=domingo
            .withColumn("nombre_mes", F.date_format("fecha", "MMMM"))
            .withColumn("nombre_dia", F.date_format("fecha", "EEEE"))
            .withColumn("es_fin_de_semana", F.when(F.col("dia_semana_iso") >= 6, 1).otherwise(0))
            .withColumn("primer_dia_mes", F.trunc("fecha", "month"))
            .withColumn("ultimo_dia_mes", F.last_day("fecha"))
            .withColumn("primer_dia_trimestre",
                        F.expr("make_date(anio, ((trimestre-1)*3)+1, 1)"))
            .withColumn("primer_dia_anio", F.trunc("fecha", "year"))
        )

    # --- Orden de columnas
    base_cols = ["id_tiempo", "fecha", "anio", "mes", "dia", "trimestre"]
    extra_cols = [
        "anio_mes", "semana_iso", "dia_semana_iso", "nombre_mes", "nombre_dia",
        "es_fin_de_semana", "primer_dia_mes", "ultimo_dia_mes",
        "primer_dia_trimestre", "primer_dia_anio"
    ]
    ordered = base_cols + [c for c in extra_cols if c in df_dim.columns]

    return df_dim.select(*ordered)


In [0]:
# Si tu DF tiene columna "fec_alta"
df_dimtiempo = crear_dim_tiempo(pedido,"FECPEDID")

# Forzar rango manualmente
df_dimtiempo = crear_dim_tiempo(df_clientes, "fec_alta",
                            start_date="2020-01-01", end_date="2025-12-31",
                             include_extras=True)


In [0]:
df_dimtiempo.show()

In [0]:
%sql
USE CATALOG desarrollo;

SHOW DATABASES;

In [0]:
from typing import List, Optional
from delta.tables import DeltaTable

def crear_tabla_delta_merge_managed(
    nombre_df: str,
    nombre_tabla: str,
    llave_origen: List[str],
    llave_destino: List[str],
    db_name: str = "default",
    catalog_name: str = "desarrollo",
    partition_cols: Optional[List[str]] = None,
    auto_merge_schema: bool = True
) -> None:
    """
    Crea si no existe una tabla Delta GESTIONADA en la base (que ya debe tener LOCATION en tu mount)
    y realiza MERGE. No usa LOCATION explícito.
    """

    # Validaciones
    df = globals()[nombre_df]

    if len(llave_origen) != len(llave_destino):
        print("❌ Error: La cantidad de columnas en 'llave_origen' y 'llave_destino' no coinciden.")
        return

    if partition_cols:
        faltantes = [c for c in partition_cols if c not in df.columns]
        if faltantes:
            print(f"❌ Error: Columnas de partición no existen en el DataFrame: {faltantes}")
            return

    if auto_merge_schema:
        spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    # Armar nombre completo
    full_name = f"{catalog_name}.{db_name}.{nombre_tabla}"

    # ✅ FIX: usar el overload moderno (una sola cadena)
    exists = spark.catalog.tableExists(full_name)

    if not exists:
        # Crear como TABLA GESTIONADA en el LOCATION de la DB (sin LOCATION explícito)
        writer = df.write.format("delta").mode("overwrite")
        if partition_cols:
            # ✅ FIX: varargs
            writer = writer.partitionBy(*partition_cols)
        writer.saveAsTable(full_name)
        print(f"✅ Tabla gestionada creada: {full_name} (bajo LOCATION de la base '{db_name}')")
        return

    # Si existe, MERGE
    try:
        delta_tbl = DeltaTable.forName(spark, full_name)
    except Exception as e:
        raise RuntimeError(f"❌ La tabla {full_name} no es Delta o no es accesible como Delta: {e}")

    merge_condition = " AND ".join(
        [f"tgt.`{llave_destino[i]}` = src.`{llave_origen[i]}`" for i in range(len(llave_origen))]
    )
    set_expr  = {c: f"src.`{c}`" for c in df.columns}
    vals_expr = {c: f"src.`{c}`" for c in df.columns}

    print(f"🔄 Ejecutando MERGE INTO {full_name} ...")
    (delta_tbl.alias("tgt")
             .merge(df.alias("src"), merge_condition)
             .whenMatchedUpdate(set=set_expr)
             .whenNotMatchedInsert(values=vals_expr)
             .execute())
    print(f"✅ MERGE completado para {full_name}")

In [0]:
# Ejecutar la función para crear la tabla y hacer MERGE usando diferentes llaves
crear_tabla_delta_merge_managed(
    nombre_df="moneda",
    nombre_tabla="dim_moneda",
    llave_origen=["id_moneda"],
    llave_destino=["id_moneda"],
    db_name="gold_ventas",
    partition_cols=["fecha_carga"]  # opcional; si no quieres partición, quítalo
)


In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Dimproducto",
    nombre_tabla="dim_producto",
    llave_origen=["id_producto"],
    llave_destino=["id_producto"],
    db_name="gold_ventas",
    partition_cols=["fecha_carga"]  # opcional; si no quieres partición, quítalo
)

In [0]:
# crear_tabla_delta_merge_managed(
#     nombre_df="df_DimCliente",
#     nombre_tabla="dim_cliente",
#     llave_origen=["id_cliente"],
#     llave_destino=["id_cliente"],
#     db_name="gold_ventas",
#     partition_cols=["fecha_carga"]  # opcional; si no quieres partición, quítalo
# )

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="vendedor",
    nombre_tabla="dim_vendedor",
    llave_origen=["id_vendedor"],
    llave_destino=["id_vendedor"],
    db_name="gold_ventas",
    partition_cols=["fecha_carga"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="mediopago",
    nombre_tabla="dim_mediopago",
    llave_origen=["id_mediopago"],
    llave_destino=["id_mediopago"],
    db_name="gold_ventas",
    partition_cols=["fecha_carga"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="prioridadpedido",
    nombre_tabla="dim_prioridadpedido",
    llave_origen=["id_prioridadpedido"],
    llave_destino=["id_prioridadpedido"],
    db_name="gold_ventas",
    partition_cols=["fecha_carga"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="modalidadenvio",
    nombre_tabla="dim_modalidadenvio",
    llave_origen=["id_modalidadenvio"],
    llave_destino=["id_modalidadenvio"],
    db_name="gold_ventas",
    partition_cols=["fecha_carga"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="modalidadventa",
    nombre_tabla="dim_modalidadventa",
    llave_origen=["id_modalidadventa"],
    llave_destino=["id_modalidadventa"],
    db_name="gold_ventas",
    partition_cols=["fecha_carga"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="pedido",
    nombre_tabla="fact_pedido",
    llave_origen=["id_pedido"],
    llave_destino=["id_pedido"],
    db_name="gold_ventas",
    partition_cols=["fecha_carga"]  # opcional; si no quieres partición, quítalo
)

In [0]:
df_dimtiempo:pyspark.sql.connect.dataframe.DataFrame
id_tiempo:integer
fecha:date
anio:integer
mes:integer
dia:integer

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_dimtiempo",
    nombre_tabla="dim_tiempo",
    llave_origen=["df_dimtiempo"],
    llave_destino=["df_dimtiempo"],
    db_name="gold_ventas"
)